# Imports

In [ ]:
## Import necessary libraries here
import os
import random
import cv2
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
from google.colab.patches import cv2_imshow
import itertools
import copy
from math import sqrt
import math
from scipy.linalg import sqrtm
import plotly.graph_objects as go

# Image stitching

<img src="https://drive.google.com/uc?id=1ev7ax-ujoOAdfy4a5XqlERXTeSOX1WQq" width="800"/>

## Overview

In this problem, you will implement an algorithm for automatically estimating the fundamental matrix F using RANSAC and the normalized 8-point algorithm. 

Image Stitching Algorithm Overview
1. Detect keypoints
2. Match keypoints
3. Estimate homography with matched keypoints (using RANSAC)
4. Combine images

**Note:**  Do not use existing image stitching code, such as found on the web, and OpenCV.

## Data

**WARNING: Colab deletes all files everytime runtime is disconnected. Make sure to re-download the inputs when it happens.**

In [ ]:
# Download Data -- run this cell only one time per runtime
!gdown 1fnD0hJ8-_Rngsc-m96ghKtdZAMf0VTjy
!unzip "/content/hill.zip" -d "/content/hill"

!gdown 1v2BFVMV0McuD5BstLvDmo1U9MrFAByS5
!unzip "/content/tv.zip" -d "/content/tv"


## Helper Functions

In [ ]:
def plot_matches(img1, img2, keypoints_1, keypoints_2, matches):
  combined_img = np.hstack((img1, img2))
  img1_cols = img1.shape[1]
  colors = [(0, 0, 255), (0, 255,0), (255,0,0), (255,255,0), (0,255,255), (255,0,255)]
  j=0
  for i in range(len(matches)):
    pt1 = (round(tuple(keypoints_1[matches[i][0]].pt)[0])          , round(tuple(keypoints_1[matches[i][0]].pt)[1]))
    pt2 = (round(tuple(keypoints_2[matches[i][1]].pt)[0])+img1_cols, round(tuple(keypoints_2[matches[i][1]].pt)[1]))
    j %= len(colors)
    color = colors[j]
    j+=1
    combined_img = cv2.line(combined_img, pt1, pt2, color, 1)
  cv2_imshow(combined_img)

def est_homography(src, dest):
    N = src.shape[0]
    if N != dest.shape[0]:
        raise ValueError("src and diff should have the same dimension")
    src_h = np.hstack((src, np.ones((N, 1))))
    A = np.array([np.block([[src_h[n], np.zeros(3), -dest[n, 0] * src_h[n]],
                            [np.zeros(3), src_h[n], -dest[n, 1] * src_h[n]]])
                  for n in range(N)])
    A = A.reshape(2 * N, 9)
    [_, _, V] = np.linalg.svd(A)
    return V.T[:, 8].reshape(3, 3)

def apply_homography(H, src):
    src_h = np.hstack((src, np.ones((src.shape[0], 1))))
    dest =  src_h @ H.T
    if dest[:,[2]].any() == 0:
      print('divide by zero')
    return (dest / dest[:,[2]])[:,0:2]

### Code

In [ ]:
def compare(x):
  return x[0]

def compute_2nn_ratio(Descriptor1, Descriptor2):
  diff = Descriptor2 - Descriptor1
  dist = np.expand_dims(np.linalg.norm(diff, axis=1), axis=1)
  temp = np.expand_dims([t for t in range(dist.shape[0])], axis=1)
  dist = np.concatenate((dist, temp), axis=1).tolist()
  dist.sort(key=compare)
  return dist[0][0]/dist[1][0], int(dist[0][1])

def detect_keypoints_and_find_matches(img1, img2, display):
  sift = cv2.xfeatures2d.SIFT_create()
  keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
  keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
  matches = []
  threshold_2NN = 0.45 #0.45
  for i in range(len(keypoints_1)):
    ratio, closest_match = compute_2nn_ratio(descriptors_1[i], descriptors_2)
    if ratio < threshold_2NN:
      matches.append([i, closest_match])
  print('Number of matches =', len(matches))
  if display:
    plot_matches(img1, img2, keypoints_1, keypoints_2, matches)
  return matches, keypoints_1, keypoints_2

def normalize__(img1_shape, img2_shape, matches, keypoints_1, keypoints_2):
  x1 = [i for i in range(img1_shape[1])]
  y1 = [i for i in range(img1_shape[0])]
  x2 = [i for i in range(img2_shape[1])]
  y2 = [i for i in range(img2_shape[0])]
  (mean_x1, std_x1, mean_y1, std_y1) = [np.mean(x1), np.std(x1), np.mean(y1), np.std(y1)]
  (mean_x2, std_x2, mean_y2, std_y2) = [np.mean(x2), np.std(x2), np.mean(y2), np.std(y2)]
  T1 = np.array([[1/std_x1,     0   , -mean_x1/std_x1],
                [    0    , 1/std_y1, -mean_y1/std_y1],
                [    0    ,     0   ,        1       ]])
  T2 = np.array([[1/std_x2,     0   , -mean_x2/std_x2],
                [    0    , 1/std_y2, -mean_y2/std_y2],
                [    0    ,     0   ,        1       ]])
  
  normalized_coords1 = []
  normalized_coords2 = []
  for i in range(len(matches)):
    (temp1, temp2) = [list(keypoints_1[matches[i][0]].pt), list(keypoints_2[matches[i][1]].pt)]
    temp1.append(1)
    temp2.append(1)
    (temp1, temp2) = [np.array(temp1), np.array(temp2)]
    temp1 = T1 @ temp1
    temp2 = T2 @ temp2
    normalized_coords1.append(temp1)
    normalized_coords2.append(temp2)

  normalized_coords1 = np.array(normalized_coords1)
  normalized_coords2 = np.array(normalized_coords2)
  xy1_ = normalized_coords1[:, :-1]
  xy2_ = normalized_coords2[:, :-1]
  return xy1_, xy2_, T1, T2


def stitch_2_images(img1, img2, display):
  matches, keypoints_1, keypoints_2 = detect_keypoints_and_find_matches(img1, img2, display)
  xy1_, xy2_, T1, T2 = normalize__(img1.shape, img2.shape, matches, keypoints_1, keypoints_2)
  distance_threshold = 0.01 #0.01
  max_num_inliers = -1
  for _ in range(1000):
    rand_indices = random.sample(range(0, xy1_.shape[0]), 4)
    subset_xy2_ = xy2_[tuple([rand_indices])]
    subset_xy1_ = xy1_[tuple([rand_indices])]
    H_norm = est_homography(subset_xy1_, subset_xy2_)
    xy1_homo = apply_homography(H_norm, xy1_)
    diff = xy2_ - xy1_homo
    distances = np.linalg.norm(diff, axis=1)
    distances[distances<=distance_threshold] = -1
    distances[distances>distance_threshold] = 0
    num_inliers = np.sum(distances) / -1
    if max_num_inliers < num_inliers:
      max_num_inliers = num_inliers
      max_inlier_comb = tuple(np.arange(xy2_.shape[0])[distances==-1])

  print('Max number of inliers =', max_num_inliers)
  subset_xy1_ = xy1_[tuple([max_inlier_comb])]
  subset_xy2_ = xy2_[tuple([max_inlier_comb])]
  H_norm = est_homography(subset_xy1_, subset_xy2_)
  H = np.linalg.inv(T2) @ H_norm @ T1
  print('De-normalized Homography:\n', H)
  xy1 = np.mgrid[0:img1.shape[1]:0.25, 0:img1.shape[0]:0.25]
  xy1 = xy1.reshape((2, -1))
  xy1 = np.transpose(xy1)

  xy1_homo = apply_homography(H, xy1)
  combined_img = np.zeros((img1.shape[0], img1.shape[1]+img2.shape[1], 3)) # define the size of new canvas
  combined_img[:, -img2.shape[1]:, :] = img2
  counter_rows = 0
  counter_cols_L = 0
  counter_cols_R = 0
  counter_rows_cols = 0
  for i in range(xy1_homo.shape[0]):
    new_row = int(xy1_homo[i][1])
    new_col = int(xy1_homo[i][0]+img1.shape[1])
    if (new_row < 0) or (new_row > img1.shape[0]-1) or (new_col > img1.shape[1]+img2.shape[1]-1) or (new_col < 0):
      continue
    combined_img[new_row, new_col] = img1[int(xy1[i][1]), int(xy1[i][0])]
  combined_img = combined_img.astype(np.uint8)
  if display:
    cv2_imshow(combined_img)
  return combined_img


# img1 = cv2.imread('/content/tv/1.jpg')
# img2 = cv2.imread('/content/tv/2.jpg')
# img3 = cv2.imread('/content/tv/3.jpg')
img1 = cv2.imread('/content/hill/1.JPG')
img2 = cv2.imread('/content/hill/2.JPG')
img3 = cv2.imread('/content/hill/3.JPG')
stitched_12 = stitch_2_images(img1, img2, True)
stitched_123 = stitch_2_images(stitched_12, img3, True)

### Write-up

- 1) Removing incorrect matches with RANSAC:
  - RANSAC involves rejecting outliers. It does this by randomly sampling N samples from M (M>N) data points (here, match pairs) & fitting an equation to those N samples. Then, all the M data points are substituted in the above fitted equation & those data points which don't satisfy the equation upto a threshold distance are considered as incorrect matches (outliers) & they are removed.
  - This process is repeated for many iterations & the iteration which has the least number of outliers is selected to get the final estimate for the equation.

- 2) The following are the de-normalized Homography matrices for stitching (i) the 1st hill image with the 2nd hill image & (ii) the stitched 1st & 2nd hill image with the 3rd hill image:
  - <img src="https://drive.google.com/uc?id=1z0ipA7VsynFlsna9DXdj_0Q5Lor8tsRu" width="400"/>
  - <img src="https://drive.google.com/uc?id=1OAQtWGj9hDfdVMkJfm6a8GpRSPw4b-8V" width="400"/>

- 3) Blended images:
  - <img src="https://drive.google.com/uc?id=1zVbOac1xOpr8WYGUfwRDfqBL9ozW4KW5"/>
  - <img src="https://drive.google.com/uc?id=1qTLNd6yPPMlEbP5xhaOlQz01Tsr7zgGd"/>